In [1]:
#!pip install tensorflow
#!pip install sklearn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# Load the BMI dataset
data = pd.read_csv("data.csv")
data.head()

,Unnamed: 0,bmi,gender,is_training,name
0,0,34.207396,Male,1,img_0.bmp
1,1,26.453720,Male,1,img_1.bmp
2,2,34.967561,Female,1,img_2.bmp
3,3,22.044766,Female,1,img_3.bmp
4,4,37.758789,Female,1,img_4.bmp


In [4]:
print(len(data))

4206


In [5]:
print(len(data[data['is_training']==1]))
print(len(data[data['is_training']==0]))

3368
838


In [6]:
data.info()
image_paths='C:/Users/kisho/Desktop/UChicago Academics/Images'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4206 entries, 0 to 4205
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   4206 non-null   int64  
 1   bmi          4206 non-null   float64
 2   gender       4206 non-null   object 
 3   is_training  4206 non-null   int64  
 4   name         4206 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 164.4+ KB


In [7]:
data.value_counts()

Unnamed: 0  bmi        gender  is_training  name        
0           34.207396  Male    1            img_0.bmp       1
2793        28.662354  Female  1            img_2793.bmp    1
2795        26.289704  Female  1            img_2795.bmp    1
2796        33.792661  Male    1            img_2796.bmp    1
2797        28.160551  Male    1            img_2797.bmp    1
                                                           ..
1407        46.511695  Female  1            img_1407.bmp    1
1408        26.622856  Male    1            img_1408.bmp    1
1409        24.900200  Male    1            img_1409.bmp    1
1410        50.029844  Male    1            img_1410.bmp    1
4205        34.618844  Male    0            img_4205.bmp    1
Length: 4206, dtype: int64

In [8]:
# Preprocess the image paths and labels
image_paths = data["name"].values

valid_image_paths = []
import os
missing_images=0
missing_list = []
for image_path in image_paths:
    if os.path.isfile(image_path):
        valid_image_paths.append(image_path)
    else:
        missing_images+=1
        missing_list.append(image_path)
        #print(f"Image file not found: {image_path}")
        
print(missing_images)
#print(missing_list)
#image_paths = valid_image_paths
#labels = data["bmi"].values

244


In [9]:
data_filtered = data[~data['name'].isin(missing_list)]

In [10]:
data = data_filtered
len(data)

3962

In [11]:
print(len(data[data['is_training']==1]))
print(len(data[data['is_training']==0]))

3210
752


In [12]:
train_data = data[data["is_training"] == 1]
test_data = data[data["is_training"] == 0]

In [13]:
train_paths = train_data["name"].tolist()
train_labels = train_data["bmi"].tolist()

val_paths = test_data["name"].tolist()
val_labels = test_data["bmi"].tolist()

#from sklearn.model_selection import train_test_split

#train_paths, val_paths, train_labels, val_labels = train_test_split(
#    train_paths, train_labels, test_size=0.2, random_state=42
#)

In [14]:
print(len(train_labels))
print(len(val_labels))

3210
752


#### Adding Custom Layers: The code adds custom layers on top of the VGG16 model. It adds a global average pooling layer to reduce the spatial dimensions, followed by a fully connected (dense) layer with ReLU activation function, and an output layer with a single neuron for BMI prediction.

In [15]:
from tensorflow.keras.layers import Dropout

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Create a new model for feature extraction
output_layer = base_model.get_layer("block5_conv3")
model = Model(inputs=base_model.input, outputs=output_layer.output)

# Unfreeze the last 5 layers of the pre-trained VGG16 model
for layer in base_model.layers[-5:]:
    layer.trainable = True

# Add a fully connected layer for classification
x = GlobalAveragePooling2D()(output_layer.output)
x = Dense(64, activation="relu")(x)
x = Dense(128, activation="relu")(x)

# Add a Dropout layer with a dropout rate of 0.5
x = Dropout(0.3)(x)
predictions = Dense(1)(x)

# Create the final model
model = Model(inputs=model.input, outputs=predictions)
model.compile(optimizer="adam", loss="mean_squared_error")

In [16]:
%%time 

# Define the image data generator
#datagen = image.ImageDataGenerator(rescale=1.0 / 255.0)

# This is to use preprocessing techniques to the images to improve the performance
# Define the image data generator with data augmentation
datagen = image.ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Create training data generator with augmentation
train_generator = datagen.flow_from_dataframe(
    pd.DataFrame({"path": train_paths, "BMI": train_labels}),
    x_col="path",
    y_col="BMI",
    target_size=(224, 224),
    batch_size=32,
    class_mode="raw",
)

# Create training and validation data generators
#train_generator = datagen.flow_from_dataframe(
#    pd.DataFrame({"path": train_paths, "BMI": train_labels}),
#    x_col="path",
#    y_col="BMI",
#    target_size=(224, 224),
#    batch_size=32,
#    class_mode="raw",
#)

val_generator = datagen.flow_from_dataframe(
    pd.DataFrame({"path": val_paths, "BMI": val_labels}),
    x_col="path",
    y_col="BMI",
    target_size=(224, 224),
    batch_size=32,
    class_mode="raw",
)

# Train the model
#history = model.fit(train_generator, validation_data=val_generator, epochs=2)

# Define a learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3)

# Train the model with learning rate scheduler
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[lr_scheduler]
)

Found 3210 validated image filenames.
Found 752 validated image filenames.
Epoch 1/5
101/101 [==============================] - 1103s 11s/step - loss: 904.1679 - val_loss: 115.0631 - lr: 0.0010
Epoch 2/5
101/101 [==============================] - 1063s 11s/step - loss: 110.6997 - val_loss: 124.5840 - lr: 0.0010
Epoch 3/5
101/101 [==============================] - 2214s 22s/step - loss: 104.9940 - val_loss: 134.5201 - lr: 0.0010
Epoch 4/5
101/101 [==============================] - 1009s 10s/step - loss: 89.7133 - val_loss: 91.9335 - lr: 0.0010
Epoch 5/5
101/101 [==============================] - 1079s 11s/step - loss: 93.3787 - val_loss: 104.6118 - lr: 0.0010
CPU times: total: 5h 44min 23s
Wall time: 1h 47min 48s


In [17]:
# Save the model
model.save("bmi_mode_2.h5")

In [18]:
%%time

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# Calculate predictions for the validation set
val_predictions = model.predict(val_generator)
val_predictions = val_predictions.flatten()

# Create a dataframe for actual and predicted BMI values
val_results = pd.DataFrame({'Actual BMI': val_labels, 'Predicted BMI': val_predictions})

# Add the image file names column
val_results['Image File'] = val_paths

# Define the BMI categories
bmi_categories = ["Underweight", "Normal Weight", "Overweight"]

# Convert actual BMI values to BMI categories
val_results['Actual BMI Category'] = pd.cut(val_results['Actual BMI'], bins=[0, 18.5, 25, np.inf], labels=bmi_categories)

# Convert predicted BMI values to BMI categories
val_results['Predicted BMI Category'] = pd.cut(val_results['Predicted BMI'], bins=[0, 18.5, 25, np.inf], labels=bmi_categories)

# Calculate accuracy
val_accuracy = accuracy_score(val_results['Actual BMI Category'], val_results['Predicted BMI Category'])

print("Validation Accuracy:", val_accuracy)
print(val_results)

24/24 [==============================] - 45s 2s/step
Validation Accuracy: 0.8404255319148937
     Actual BMI  Predicted BMI    Image File Actual BMI Category  \
0     29.698495      29.799629  img_3369.bmp          Overweight   
1     30.845918      30.087578  img_3370.bmp          Overweight   
2     24.389796      29.673561  img_3371.bmp       Normal Weight   
3     36.258679      31.547768  img_3372.bmp          Overweight   
4     27.891291      30.126839  img_3373.bmp          Overweight   
..          ...            ...           ...                 ...   
747   34.078947      29.585171  img_4201.bmp          Overweight   
748   34.564776      30.161367  img_4202.bmp          Overweight   
749   27.432362      28.966759  img_4203.bmp          Overweight   
750   40.492800      31.033522  img_4204.bmp          Overweight   
751   34.618844      28.541029  img_4205.bmp          Overweight   

    Predicted BMI Category  
0               Overweight  
1               Overweight  
2  

In [19]:
print("Minimum Predicted BMI:", np.min(val_predictions))
print("Maximum Predicted BMI:", np.max(val_predictions))

Minimum Predicted BMI: 26.151342
Maximum Predicted BMI: 34.820595


In [20]:
val_results.head()

,Actual BMI,Predicted BMI,Image File,Actual BMI Category,Predicted BMI Category
0,29.698495,29.799629,img_3369.bmp,Overweight,Overweight
1,30.845918,30.087578,img_3370.bmp,Overweight,Overweight
2,24.389796,29.673561,img_3371.bmp,Normal Weight,Overweight
3,36.258679,31.547768,img_3372.bmp,Overweight,Overweight
4,27.891291,30.126839,img_3373.bmp,Overweight,Overweight


In [21]:
val_results.to_csv('output_bmi.csv', index=False)

In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate predictions for the validation set
val_predictions = model.predict(val_generator)
val_predictions = val_predictions.flatten()

# Calculate RMSE
val_rmse = np.sqrt(mean_squared_error(val_labels, val_predictions))

# Calculate MAE
val_mae = mean_absolute_error(val_labels, val_predictions)

print("Validation RMSE:", val_rmse)
print("Validation MAE:", val_mae)

24/24 [==============================] - 44s 2s/step
Validation RMSE: 10.18339332012163
Validation MAE: 7.386894663989609


In [23]:
from sklearn.metrics import roc_auc_score

# Convert actual BMI values to binary labels (0 or 1)
actual_labels = pd.cut(val_results['Actual BMI'], bins=[0, 24.9, np.inf], labels=[0, 1])

# Calculate AUC
auc = roc_auc_score(actual_labels, val_results['Predicted BMI'])

# Print AUC
print("AUC:", auc)

AUC: 0.45721508349598783


In [24]:
from sklearn.metrics import accuracy_score, r2_score

# Calculate R2 score
val_r2 = r2_score(val_results['Actual BMI'], val_results['Predicted BMI'])
print("R2 Score:",val_r2)

R2 Score: -0.22980796501967649


In [25]:
import pandas as pd

# Assuming you have a DataFrame called 'df'
# ...

# Save the DataFrame as a CSV file
val_results.to_csv('predictions_trial.csv', index=False)
